Demographics    

In [24]:
import pandas as pd 
from scipy import stats
from statsmodels.stats.multitest import multipletests
import numpy as np
from pathlib import Path
Path.cwd()

WindowsPath('c:/Users/juhoffmann/Desktop/SubliminalVideoPriming/scripts/demographics')

In [25]:
data_df = Path("..") / ".." / "data" / "demographics" / "all.xlsx"
df = pd.read_excel(data_df)
df.head()

,EmoCon_ID,ID,Video Mask done,Gender_x,Alter,Gender_f1_m2,Group_MDD1_HC2,BVAQ,BDI-II,STAI2,...,CERQ_Refokussierung auf Planung,CERQ_Positive Neubewertung,CERQ_Relativieren,CERQ_Katastrophisierung,CERQ_Andere Beschuldigen,Antidepressants,Patients_HC,Gender_y,Medication,AD_Classes
0,EmoCon006,Sub-004,1,male,29,2,2,56,6,47,...,17,13,15,7,7,NaN,NaN,NaN,NaN,NaN
1,EmoCon007,Sub-006,1,female,23,1,2,47,3,42,...,14,16,17,7,10,NaN,NaN,NaN,NaN,NaN
2,EmoCon001,Sub-007,1,male,23,2,1,52,26,69,...,8,4,7,15,4,1.0,Patient,male,morning: Citalopram 30mg,['SSRI']
3,EmoCon017,Sub-008,1,female,19,1,1,58,35,68,...,13,11,5,9,7,1.0,Patient,female,morning: Escitalopram 10mg evening: Quetiapin...,['SSRI']
4,EmoCon015,Sub-009,1,male,27,2,1,59,14,68,...,12,5,4,14,7,1.0,Patient,male,morning: Elontril 300mg,['NDRI']


In [26]:
df = df.rename(columns={
    "ID": "Participant_ID",
    "Video Mask done": "VideoMask_Completed",
    "Gender_x": "Gender",
    "Gender_y": "Gender_Check",
    "Alter": "Age",
    "Gender_f1_m2": "Gender_F1_M2",
    "Group_MDD1_HC2": "Group_MDD1_HC2",
    "BVAQ": "BVAQ_Total",
    "BDI-II": "BDI_II_Total",
    "STAI1": "STAI_State",
    "STAI2": "STAI_Trait",
    "WMS Correct": "WMS_Correct",
    "TM A": "TrailMaking_A_Time",
    "TM B": "TrailMaking_B_Time",
    "Error A": "TrailMaking_A_Errors",
    "Error B": "TrailMaking_B_Errors",
    "Hamilton Score Norm": "HAMD_Diagnose",
    "Hamilton Score": "HAMD_Total",
    "DigitSpan_Forwards_Span": "DigitSpan_Forward_Span",
    "DigitSpan_ForwardsTotal Score": "DigitSpan_Forward_Total",
    "DigitSpan_Backwards_Span": "DigitSpan_Backward_Span",
    "DigitSpan_Backwards_Total Score": "DigitSpan_Backward_Total",
    "DERS_Gesamtwert": "DERS_Total",
    "DERS_Nicht-Akzeptanz emotionalerReaktionen": "DERS_NonAcceptance",
    "DERS_Probleme mit zielorientiertem Verhalten": "DERS_GoalDifficulties",
    "DERS_Impulskontrollprobleme": "DERS_ImpulseControl",
    "DERS_Mangel an emotionaler Aufmerksamkeit": "DERS_LackOfAwareness",
    "DERS_Eingeschränkter Zugang zu Emotionsregulations-Strategien": "DERS_LimitedStrategies",
    "DERS_Mangel an emotionaler Klarheit": "DERS_LackOfClarity",
    "CERQ_Selbstbeschuldigung": "CERQ_SelfBlame",
    "CERQ_Akzeptanz": "CERQ_Acceptance",
    "CERQ_Rumination": "CERQ_Rumination",
    "CERQ_Positive Refokussierung": "CERQ_PositiveRefocusing",
    "CERQ_Refokussierung auf Planung": "CERQ_RefocusingOnPlanning",
    "CERQ_Positive Neubewertung": "CERQ_PositiveReappraisal",
    "CERQ_Relativieren": "CERQ_PuttingIntoPerspective",
    "CERQ_Katastrophisierung": "CERQ_Catastrophizing",
    "CERQ_Andere Beschuldigen": "CERQ_BlameOthers",
    "Antidepressants": "Antidepressant_Use",
    "Medication": "Medication_Status",
    "AD_Classes": "Antidepressant_Class"
})

In [27]:
df.columns  

Index(['EmoCon_ID', 'Participant_ID', 'VideoMask_Completed', 'Gender', 'Age',
       'Gender_F1_M2', 'Group_MDD1_HC2', 'BVAQ_Total', 'BDI_II_Total',
       'STAI_Trait', 'STAI_State', 'WMS_Correct', 'TrailMaking_A_Time',
       'TrailMaking_B_Time', 'TrailMaking_A_Errors', 'TrailMaking_B_Errors',
       'HAMD_Diagnose', 'HAMD_Total', 'DigitSpan_Forward_Span',
       'DigitSpan_Forward_Total', 'DigitSpan_Backward_Span',
       'DigitSpan_Backward_Total', 'DERS_Total', 'DERS_NonAcceptance',
       'DERS_GoalDifficulties', 'DERS_ImpulseControl', 'DERS_LackOfAwareness',
       'DERS_LimitedStrategies', 'DERS_LackOfClarity', 'CERQ_SelfBlame',
       'CERQ_Acceptance', 'CERQ_Rumination', 'CERQ_PositiveRefocusing',
       'CERQ_RefocusingOnPlanning', 'CERQ_PositiveReappraisal',
       'CERQ_PuttingIntoPerspective', 'CERQ_Catastrophizing',
       'CERQ_BlameOthers', 'Antidepressant_Use', 'Patients_HC', 'Gender_Check',
       'Medication_Status', 'Antidepressant_Class'],
      dtype='object')

Descriptive Statistics and Group Differences

In [31]:
df["Group_MDD1_HC2"] = pd.to_numeric(df["Group_MDD1_HC2"], errors="coerce")
cols = [
    "TrailMaking_A_Time",
    "TrailMaking_B_Time",
    "DigitSpan_Forward_Total",
    "DigitSpan_Backward_Total",
]
df[cols] = df[cols].apply(pd.to_numeric, errors="coerce")


exclude_cols = [
    "Antidepressant_Use", "Medication_Status", "Antidepressant_Class",
    "VideoMask_Completed", "Gender_Check", "Gender_f1_m2", "Patients_HC",
    "Gender_x", "Hamilton_Normalized", "HAMD_Diagnose"
]

numeric_cols = df.select_dtypes(include="number").columns
numeric_cols = [c for c in numeric_cols if c not in (["Group_MDD1_HC2"] + exclude_cols)]

# --- Part A: Group comparisons (MDD vs HC) for numeric variables ---
results = []

for col in numeric_cols:
    g1 = df[df["Group_MDD1_HC2"] == 1][col].dropna()  # MDD
    g2 = df[df["Group_MDD1_HC2"] == 2][col].dropna()  # HC

    if len(g1) > 1 and len(g2) > 1:
        mean1, sd1 = g1.mean(), g1.std()
        mean2, sd2 = g2.mean(), g2.std()
        median1, median2 = g1.median(), g2.median()
        range1_min, range1_max = g1.min(), g1.max()
        range2_min, range2_max = g2.min(), g2.max()

        t_stat, p_val = stats.ttest_ind(g1, g2, equal_var=False)

        # Cohen's d (pooled SD)
        n1, n2 = len(g1), len(g2)
        s1, s2 = np.var(g1, ddof=1), np.var(g2, ddof=1)
        pooled_sd = np.sqrt(((n1 - 1) * s1 + (n2 - 1) * s2) / (n1 + n2 - 2))
        cohen_d = (mean1 - mean2) / pooled_sd if pooled_sd != 0 else np.nan

        results.append({
            "Variable": col,
            "MDD Mean ± SD": f"{mean1:.2f} ± {sd1:.2f}",
            "HC Mean ± SD": f"{mean2:.2f} ± {sd2:.2f}",
            "MDD Median": f"{median1:.2f}",
            "HC Median": f"{median2:.2f}",
            "MDD Range": f"{range1_min:.2f}, {range1_max:.2f}",
            "HC Range": f"{range2_min:.2f}, {range2_max:.2f}",
            "t-statistic": t_stat,
            "p-value": p_val,
            "Cohen's d": cohen_d
        })

results_df = pd.DataFrame(results)

# Bonferroni correction (only numeric p-values)
if not results_df.empty:
    reject, p_adj, _, _ = multipletests(results_df["p-value"], alpha=0.05, method="bonferroni")
    results_df["p-value (Bonferroni)"] = np.round(p_adj, 6)
    results_df["Significant (Bonferroni<0.05)"] = reject

# --- Part B: MDD-only descriptives for medication + HAMD variables ---
mdd = df[df["Group_MDD1_HC2"] == 1].copy()
df["HAMD_Total"] = pd.to_numeric(df["HAMD_Total"], errors="coerce")

mdd_only_vars = [
    "HAMD_Total",
    "Antidepressant_Use",
    "Antidepressant_Class"
]

mdd_desc_rows = []

for var in mdd_only_vars:
    if var not in mdd.columns:
        continue

    series = mdd[var].dropna()
    if series.empty:
        continue

    # Numeric variable -> mean/SD etc.
    if pd.api.types.is_numeric_dtype(series):
        mdd_desc_rows.append({
            "Variable": var,
            "Type": "numeric",
            "N": len(series),
            "Mean ± SD": f"{series.mean():.3f} ± {series.std():.3f}",
            "Median": f"{series.median():.3f}",
            "Range": f"{series.min():.3f}, {series.max():.3f}",
            "Level": "—",
            "Count": "—",
            "Percent": "—"
        })

    # Categorical variable -> counts + %
    else:
        counts = series.value_counts(dropna=False)
        perc = series.value_counts(normalize=True, dropna=False) * 100

        for level in counts.index:
            mdd_desc_rows.append({
                "Variable": var,
                "Type": "categorical",
                "N": len(series),
                "Mean ± SD": "—",
                "Median": "—",
                "Range": "—",
                "Level": str(level),
                "Count": int(counts[level]),
                "Percent": round(float(perc[level]), 1)
            })

mdd_desc_df = pd.DataFrame(mdd_desc_rows)

print("Tables ready:")
print("Group comparison table:", results_df.shape)
print("MDD-only descriptives table:", mdd_desc_df.shape)

# --- Save both tables ---
out_dir = Path("..") / ".." / "data" / "demographics"
out_dir.mkdir(parents=True, exist_ok=True)

results_df.to_csv(out_dir / "group_comparisons_bonferroni.csv", sep=";", index=False)
mdd_desc_df.to_csv(out_dir / "mdd_only_descriptives.csv", sep=";", index=False)

Tables ready:
Group comparison table: (27, 12)
MDD-only descriptives table: (9, 9)


In [32]:
numeric_cols

['Age',
 'Gender_F1_M2',
 'BVAQ_Total',
 'BDI_II_Total',
 'STAI_Trait',
 'STAI_State',
 'WMS_Correct',
 'TrailMaking_A_Time',
 'TrailMaking_B_Time',
 'HAMD_Total',
 'DigitSpan_Forward_Total',
 'DigitSpan_Backward_Total',
 'DERS_Total',
 'DERS_NonAcceptance',
 'DERS_GoalDifficulties',
 'DERS_ImpulseControl',
 'DERS_LackOfAwareness',
 'DERS_LimitedStrategies',
 'DERS_LackOfClarity',
 'CERQ_SelfBlame',
 'CERQ_Acceptance',
 'CERQ_Rumination',
 'CERQ_PositiveRefocusing',
 'CERQ_RefocusingOnPlanning',
 'CERQ_PositiveReappraisal',
 'CERQ_PuttingIntoPerspective',
 'CERQ_Catastrophizing',
 'CERQ_BlameOthers']